# Hi

In [5]:
# Load dependencies
# import numpy as np
from ipycanvas import Canvas,  hold_canvas
canvas = Canvas(width=800, height=600)

## The First Maze
Leverage the "recursive backtracker" algorithm. It is a randomized version of the depth-first search algorithm.

### The Recursive Algorithm
1. Given a current cell as a parameter, Mark the current cell as visited.
2. While the current cell has any unvisited neighbor cells:
  1. Choose one of the unvisited neighbors.
  2. Remove the wall between the current cell and the chosen cell.
  3. Invoke the routine recursively for a chosen cell.

### The Iterative Algorithm
To avoid a deep recursive, a stack can be used.

1. Choose the initial cell, mark it as visited and push it to the stack.
2. While the stack is not empty:
  1. Pop a cell from the stack and make it a current cell.
  2. If the current cell has any neighbors which have not been visited:
    1. Push the current cell to the stack.
    2. Choose one of the unvisited neighbors.
    3. Remove the wall between the current cell and the chosen cell.
    4. Mark the chosen cell as visited and push it to the stack.

### The Data Structures


In [6]:

with hold_canvas(canvas):
  canvas.fill_style = 'green'
  canvas.line_width = 5
  canvas.begin_path()
  canvas.move_to(canvas.width*0.2, canvas.height/2)
  canvas.line_to(canvas.width*0.6, canvas.height/2)
  canvas.stroke()
canvas

Canvas(height=600, width=800)